In [1]:
from gensim.models import KeyedVectors
import numpy as np
from joblib import dump, load
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

In [ ]:
# Usa frecuencia de palabras
def create_tf_idf(list_text):
    vectorizer = TfidfVectorizer(use_idf=False, norm=None, ngram_range=(1, 1)) # use_idf=true ponderacion de frecuancia de palabras en documentos
    tfidf = vectorizer.fit(list(list_text))
    vectores_tfidf=vectorizer.transform(list_text)
    dump(tfidf,'tfidf_model2joblib')
    print('Fue creado modelo TF-IDF')
    #vectores_tfidf=list(matrix)
    #print(len(vectores_tfidf))
    return vectores_tfidf

In [3]:
# Retorna un vector TF-IDF 
def get_vector_tfidf(list_text):
    tfidf=load('tfidf_model2.joblib')# Ojo cuidado, solo cargar solo una vez
    words_ntf=[x for x in list_text.split() if x not in tfidf.get_feature_names()] # Palabras que no estan en el vocabulario del modelo
    return tfidf.transform([list_text]).toarray()

In [4]:
# Leyendo los datos CSV
datos_notacion = pd.read_csv('natacion_suicidio.csv')
datos_general = pd.read_csv('suicidio_todo.csv') 

In [5]:
print(datos_notacion.size)
print(datos_general.size)

3957
381409


In [6]:
# seleccionamos solo los datos que tienen tendencia suicida
selec_dat_1=datos_notacion.loc[datos_notacion['suicidio'] == 1]
selec_dat_1.head(2)

,frase,tweet_clean,suicidio
7,cansado de estar solo,louis espero no estar pidiendo mucho solo quie...,1
17,cansado de estar solo,estoy super cansado cansado de la gente cansad...,1


In [7]:
datos_general.head(2)

,id,frase,idtw,tweet,nombre,fecha,tweet_clean
0,4901,dormir y nunca mas despertar,1185025213177704448,Ganas de dormir y no despertar nunca más,darafiorodri,18/10/2019 02:50,ganas de dormir y no despertar nunca más
1,4902,dormir y nunca mas despertar,1185004540707430400,RT @MarioMcmv75: Así es NUNCA MÁS NOS TRAGAMOS...,sanpascualitor,18/10/2019 01:27,así es nunca más nos tragamos y cerquita de di...


In [8]:
def sampling_k_elements(group, k=200): 
    if len(group) < k:
        return group
    return group.sample(k)

In [9]:
# Agrupamos y guardamos en un diccionario, cada elemento del diccionario es una lista
dict_general = datos_general.groupby('frase').apply(sampling_k_elements).reset_index(drop=True)
dict_general=dict_general.groupby('frase')['tweet_clean'].apply(list).to_dict()

# Extraemos todo los tweets para crear el vocabulario para TF-IDF
text_todo=list(datos_general.tweet_clean.values.astype('U'))
vectores_tfidf=create_tf_idf(text_todo) #vocubulario total

Fue creado modelo TF-IDF


In [10]:
for x in dict_general.keys():
    print(x,len(dict_general[x]))

cansado de estar solo 145
dormir y nunca mas despertar 52
hacerme daño 200
la vida es muy dura 200
la vida no tiene sentido 200
la vida no vale nada 200
lastimarme 200
listo para morir 118
matarme 200
me estoy ahogando 200
me odio a mi mismo 200
me odio yo mismo 200
me voy ahora 200
mi muerte 200
mi vida consiste en nada 11
mi vida es asi de miserable 21
mi vida no tiene sentido 200
mi vida no vale 200
morir en mi sueño 116
nada por lo que vivir 200
no quiero continuar 200
no quiero despertar 200
no quiero estar aqui 200
no quiero estar mas aqui 182
no quiero estar vivo 164
no quiero existir 200
no quiero intentarlo mas 76
no quiero seguir  200
no quiero vivir 200
no quiero vivir mas 200
no soporto mas 200
no vale la pena seguir viviendo 21
no vale la pena vivir 200
no valgo nada 200
odio mi vida 200
parar el dolor 188
pensando en el suicidio 112
podria quedarme solo dormido 1
poner fin a esto 200
por que debería seguir viviendo 16
que esta mal conmigo 200
que tengo de malo 200
querer 

In [11]:
list(selec_dat_1.frase)[95]

'no quiero seguir'

In [12]:
similares=defaultdict(list) #inicializamos un diccionario de listas para guardar la data necesaria
for i,j in zip(list(selec_dat_1.frase),list(selec_dat_1.tweet_clean)):
    vec_selec=get_vector_tfidf(j)
    print(i)
    if i in dict_general:
        for x in dict_general[i]:
            vec_todo_j=get_vector_tfidf(x)
            simi=cosine_similarity(vec_selec,vec_todo_j)[0][0]
            if simi>0.6 and simi<0.95 and x not in similares['tweet_similar']: # guardamos todos los tweets con similaridad alta, pero menor 0.95 que podria ser la misma frase
                similares['frase'].append(i)
                similares['tweet_anotado'].append(j)
                similares['tweet_similar'].append(x)
                similares['similaridad'].append(simi)
            

cansado de estar solo
cansado de estar solo
dormir y nunca mas despertar
dormir y nunca mas despertar
dormir y nunca mas despertar
dormir y nunca mas despertar
dormir y nunca mas despertar
dormir y nunca mas despertar
dormir y nunca mas despertar
dormir y nunca mas despertar
dormir y nunca mas despertar
dormir y nunca mas despertar
dormir y nunca mas despertar
dormir y nunca mas despertar
dormir y nunca mas despertar
dormir y nunca mas despertar
dormir y nunca mas despertar
dormir y nunca mas despertar
dormir y nunca mas despertar
hacerme daño
hacerme daño
hacerme daño
hacerme daño
hacerme daño
la vida no tiene sentido
la vida no tiene sentido
la vida no tiene sentido
la vida no tiene sentido
lastimarme
lastimarme
listo para morir
listo para morir
listo para morir
listo para morir
matarme
matarme
matarme
matarme
matarme
matarme
matarme
matarme
matarme
matarme
matarme
matarme
matarme
matarme
matarme
matarme
me estoy ahogando
me estoy ahogando
me estoy ahogando
me estoy ahogando
me estoy

In [13]:
# convertimos el diccionario de listas a un Dataframe e lo guardamos como CSV
findal_dt=pd.DataFrame.from_dict(similares,orient='index').transpose()
findal_dt.to_csv(r'datos_similares_notados_2.csv')

In [14]:
aaa

NameError: name 'aaa' is not defined

# Preparar data para re-anotacion

In [2]:
datos_notar1 = pd.read_csv('datos_nota_sim.csv') 
datos_notar2 = pd.read_csv('datos_similares_notados_2.csv') 

In [3]:
ya_notados=list(datos_notar1.tweet_clean)

In [4]:
no_notados=list(datos_notar2.tweet_similar)

In [5]:
len(list(set(ya_notados)))

457

In [6]:
nuevos=[]
for i in no_notados:
    if i not in ya_notados:
        nuevos.append(i)

In [7]:
len(nuevos)

298

In [8]:
xxx = pd.DataFrame({'tweet_clean':nuevos})

In [9]:
xxx.to_csv(r'mas2.csv')